In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from tensorflow import keras

from scikeras.wrappers import KerasClassifier


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(X, y)
y_proba = clf.predict_proba(X)


In [2]:
from copy import deepcopy

import sklearn
from sklearn.utils.multiclass import type_of_target

from ami.macrobot.jobspec import MBJobspec
from ami.macrobot.runner import MacrobotRunner
from macro_utils import DotDict

raw_spec = DotDict({
    "accessors_fn": "train_and_shap",
    "charting": [
        "TrainChart",
        "ShapChart"
    ],
    "data_splitter": {
        "expanding": 0,
        "initial_training_size": 350,
        "test_size": 8
    },
    "ensemble": {
        "agg_fn": None,
        "max_samples": None,
        "n_estimators": 10,
        "n_jobs": -1,
        "type": "bagging"
    },
    "features_transform": {
        "PCA": [
        ],
        "PCA_diff": [],
        "clip_level": 4,
        "data_startsfrom": "2005-06-17",
        "diff": {
            "names": [
            ],
            "transform": "make_pipeline(Difference(1), StandardScaler(),Clipper(-8, 8))"
        },
        "pass_through": {
            "names": [
                "EMU_CECI",
                "US_CECI",
                "EMU_CPI_Core",
                "FX_EUR_TR_RSI",
            ],
            "transform": "make_pipeline(StandardScaler(), Clipper(-8, 8))"
        },
        "untransformed": {
            "names": [
                "US_CPI_Core_ZSc52"
            ],
            "transform": "passthrough"
        }
    },
    "model": {
        "activation": "selu",
        "batch_size": None,
        "batchnorm": True,
        "batchnorm_trainable": True,
        "batchnorm_training": False,
        "dropout": 0.0,
        "epochs": 75,
        "lookback_periods": 26,
        "loss": "mse",
        "nodes": 8,
        "optimizer": "adam",
        "recurrent_dropout": 0.2,
        "regularizer_l1": 0.01,
        "regularizer_l2": 0.0,
        "type": "GruModel",
        "use_bias": False,
    },
    "name": "FX_EUR_TR",
    "strategy": {
        "c": 0.1,
        "pred_agg_fn": "mean",
        "t_costs": 0.0001,
        "threshold": 0,
        "type": "ThresholdSignStrategy"
    },
    "target": {
        "logreturn": True,
        "name": "FX_EUR_TR",
        "prediction_lag": 1,
        "transform": "make_pipeline(StandardScaler(with_mean=False), Clipper(-3, 3))"
    },
    "type": "MB_training",
    "windowframe": {
        "n_jobs": -1
    }
})

spec = MBJobspec(deepcopy(raw_spec)).parse()
runner = MacrobotRunner(spec)
data = runner.get_data()

import numpy as np
import pandas as pd
from tensorflow.keras import layers
X, y = data, data[raw_spec.target.name]


from scikeras.wrappers import KerasClassifier
from tensorflow.keras import regularizers
from typing import Iterable



class Test(KerasClassifier):

    def __init__(
            self,
            nodes=15,
            lookback_periods=13,
            activation="selu",
            kernel_initializer='random_uniform',
            regularizer_l1=0.05,
            regularizer_l2=0.0,
            recurrent_dropout=0.0,
            dropout=0.0,
            dense_units: Iterable = tuple(),
            batchnorm=True,
            batchnorm_trainable=True,
            batchnorm_training=False,
            epochs=100,
            batch_size=None,
            use_bias=False,
            **kwargs
    ):
        super().__init__(**kwargs)

        # for param in inspect.signature(self).parameters.values():
        #     setattr(self, param.name, param.default if param.name != 'dense_layers_list' else [])

        self.nodes = nodes
        self.lookback_periods = lookback_periods
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        self.regularizer_l1 = regularizer_l1
        self.regularizer_l2 = regularizer_l2
        self.recurrent_dropout = recurrent_dropout
        self.dropout = dropout
        self.dense_units = dense_units
        self.batchnorm = batchnorm
        self.batchnorm_training = batchnorm_training
        self.batchnorm_trainable = batchnorm_trainable
        self.batch_size = batch_size
        self.epochs = epochs
        self.use_bias = use_bias

    def _keras_build_fn(self, meta, **kwargs):
        inputs = layers.Input(shape=meta["X_shape_"][1:])  # , name='inputs')
        x = inputs
        x = layers.GRU(
            self.nodes,
            activation=self.activation,
            recurrent_dropout=self.recurrent_dropout,
            return_sequences=False,
            kernel_initializer=self.kernel_initializer,
            kernel_regularizer=regularizers.l1_l2(self.regularizer_l1, self.regularizer_l2),
            activity_regularizer=regularizers.l1(0.0),
            use_bias=self.use_bias
        )(x)

        x = layers.Dense(
            meta["n_classes_"],
            activation='softmax',
            use_bias=self.use_bias,
            kernel_initializer='random_uniform',
            # name='prediction'
        )(x)

        outputs = x





requesting: {'EMU_CECI', 'US_CPI_Core_ZSc52', 'US_CECI', 'EMU_CPI_Core', 'FX_EUR_TR', 'FX_EUR_TR_RSI'}


In [3]:
y.plot(backend='plotly')

In [6]:
for col in X:
    X[col].plot(backend='plotly')

In [7]:
X.iloc[:, 0].plot(backend='plotly')

In [11]:
X.iloc[:, 5].plot(backend='plotly')